In [12]:
import pandas as pd

carbon_policy_labels = {
    "carbon_waiting": "Lowest-\nWindow",
    "carbon_oracle":  "Lowest-\nWindow*",
    "carbon_lowest": "Lowest-\nSlot",
    "suspend-resume_oracle": "Wait AWhile",
    "suspend-resume-threshold_oracle": "Ecovisor",
    "carbon_cst_oracle": "Carbon-\nTime*",
    "carbon_cst_average": "Carbon-\nTime",
    "cost_oracle": "AllWait-\nThreshold",
    "carbon-cost_cst_average": "RES-First-\nCarbon-Time",
    "carbon-cost_waiting": "RES-First-\nLowest-\nWindow",
    "carbon-spot_cst_average": "Spot-First-\nCarbon-Time",
    "suspend-resume-spot_oracle": "Spot-First-\nWaitAwhile",
    "suspend-resume-spot-threshold_oracle": "Spot-First-\nEcovisor",
    "carbon-cost-spot_cst_average": "SPOT-RES-\nCarbon-Time"
}

# steal / copy this one from the existing GAIA notebooks
def load_task_details(cluster_type, task_trace, scheduling_policy, carbon_start_index, carbon_policy, carbon_trace, reserved, waiting_times_str):
    if cluster_type =="slurm":
        file_name = f"../results/{cluster_type}/{task_trace}/slurm-details-{scheduling_policy}-{carbon_start_index}-{carbon_policy}-{carbon_trace}-{reserved}-{waiting_times_str}.csv"             
    else:
        file_name = f"../results/{cluster_type}/{task_trace}/details-{scheduling_policy}-{carbon_start_index}-{carbon_policy}-{carbon_trace}-{reserved}-{waiting_times_str}.csv"             
    df = pd.read_csv(file_name)
    df["carbon_policy"] = carbon_policy_labels[scheduling_policy+"_"+carbon_policy]
    df["scheduling_policy"] = scheduling_policy
    df["start_index"] = carbon_start_index
    df["task_trace"] = task_trace    
    df = df[df['ID'] != -1]
    return df

In [13]:
import plotly.express as px
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pytz

import importlib.util
import sys


spec = importlib.util.spec_from_file_location("carbon", "../src/carbon.py")
foo = importlib.util.module_from_spec(spec)
sys.modules["carbon"] = foo
spec.loader.exec_module(foo)

import carbon


traces = ['phased'] # , "pai_200"

waiting_times = ["48", "6x24", "4"]

scheduling_policies = [ 
   #  ("carbon", "lowest"),
    ("carbon", "oracle"),
   #  ("carbon", "cst_average"),
    # ("suspend-resume-threshold", "oracle"),
    # ("suspend-resume-threshold", "oracle"),
    # ("suspend-resume", "oracle"),
]


carbon_trace = carbon.get_carbon_model("AU-SA", 7000, extra_columns=True)
start_date_in_carbon_trace_as_timestamp = carbon_trace.df.iloc[0]["timestamp"]

fig_carbon = px.scatter(carbon_trace.df, x='datetime', y="carbon_intensity_avg", color="carbon_intensity_avg", color_continuous_scale=px.colors.sequential.speed)

def time_to_dates(seconds_since_simulation_start) -> str:
    adjusted_timestamp = seconds_since_simulation_start + start_date_in_carbon_trace_as_timestamp
    date = datetime.fromtimestamp(adjusted_timestamp, pytz.timezone('UTC'))
    return date 

for trace in traces:
    
    for policy in scheduling_policies:
        for waiting_time in waiting_times:

            scheduling_policy = policy[0]
            carbon_policy = policy[1]
            df = load_task_details("simulation", trace, policy[0], 7000, policy[1], "AU-SA", 0, waiting_time).sort_values(by=["start_time", "length"])

            df["start_time_date"] = df["start_time"].apply(time_to_dates)
            df["submission_date"] = df["arrival_time"].apply(time_to_dates)
            # df["deadline"] = (df["arrival_time"] + (int(waiting_time) * 3600)).apply(time_to_dates)
            df["exit_time_date"] = df["exit_time"].apply(time_to_dates)

            min_date_in_trace = time_to_dates(df["start_time"].min())
            max_date_in_trace = time_to_dates(df["exit_time"].max())

            fig_gantt = px.timeline(df, x_start="start_time_date", x_end="exit_time_date", y="ID", hover_data=["start_time", "arrival_time"])

            submission_markers = []

            for row in df.itertuples(index=False):
                submission_markers.append({'type': 'line', 'x0': row.submission_date, 'x1': row.start_time_date, 'y0': row.ID, 'y1': row.ID, 'xref': 'x1', 'yref':'y1', 'line': dict(color="MediumPurple", width=2, dash="dot")})

            fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

            fig.add_trace(fig_gantt.data[0], row=1, col=1)
            fig.add_trace(fig_carbon.data[0], row=2, col=1)

            title_key = f"{scheduling_policy}_{carbon_policy}"
            title = f"{scheduling_policy}_{carbon_policy} ({carbon_policy_labels.get(title_key, '')})"

            fig.update_layout(
                title_text = f"{trace}'s scheduling via {title}, {waiting_time}",
                xaxis=dict(
                    type='date',
                ),
                xaxis2=dict(
                    type='date'
                ), 
                shapes=submission_markers
            )
            fig.update_xaxes(title_text="Date", range=[min_date_in_trace, max_date_in_trace])

            fig.update_yaxes(title_text="Job ID", fixedrange=True, row=1, col=1)
            fig.update_yaxes(title_text="Carbon intensity in gCO₂eq/kWh", fixedrange=True, row=2, col=1)

            yaxis2 = fig.layout.yaxis2

            fig.update_layout({'yaxis': {'range': [-0.5,df['ID'].max() + 1], 'tickmode': 'linear'}})
            fig.update_layout({'yaxis2': {'range': [0,0.5]}})

            fig.show()

In [14]:
# We need to create some plots for parts of the implementation
# This should not be used for a case study on code modularization

import pulp
import math
from typing import Dict
sys.path.append('../src/')

import power_consumption_profiles as pcp
from task import Task, set_waiting_times
# Plan: have an LP Problem where we determine startup and work phases

import plotly.graph_objects as go
from functools import reduce

starting = None
startup_finished = None
work = None
work_time_progressed = None
lin_function_dicts: Dict[str, Dict[str, Dict[str, pulp.LpVariable | float]]] = { }
startup_time_progressed = None

foo_phases_spec: pcp.ModelParameters = {
    'startup':   [
        {'name': 'Start Python', 'duration': 10, 'power': 50},
        {'name': 'Download Data', 'duration': 10, 'power': 70},
    ],
    'work': [
      {'name': 'High', 'duration': 50, 'power': 230.0}, 
      {'name': 'Low', 'duration': 50, 'power': 105.1}, 
    ]
}

mockPowerFunction = pcp.PowerFunction(foo_phases_spec)
set_waiting_times("24")

DEADLINE: int = 250

# The carbon trace is given in hours
seconds_carbon_trace = carbon_trace.extend(1)
seconds_carbon_trace.df = seconds_carbon_trace.df.head(DEADLINE)

# Example data
# TODO: seconds from the task-length to whatever this is now
WORK_LENGTH = int(mockPowerFunction.duration_work)  # Processing time for the job
print(f'WORK_LENGTH is {WORK_LENGTH}')

STARTUP_LENGTH = int(mockPowerFunction.duration_startup) #int(mockPowerFunction.duration_startup)  # Startup time for the job
print(f'STARTUP_LENGTH is {STARTUP_LENGTH}')

M = DEADLINE * 2

carbon_cost_at_time = seconds_carbon_trace.df['carbon_intensity_avg'].to_dict()

def define_lp_problem(use_startup: bool, dynamic_power: bool, use_progress: bool, linearize: bool, timelimit: int):
    global starting
    global startup_finished
    global work
    global work_time_progressed
    global lin_function_dicts
    global startup_time_progressed


    # Define the problem
    prob = pulp.LpProblem("StopResumeCarbonAwareScheduling", pulp.LpMinimize)

    if (use_startup): 

        starting = pulp.LpVariable.dicts("starting", (t for t in range(DEADLINE)), cat="Binary")
        startup_finished = pulp.LpVariable.dicts("start", (t for t in range(DEADLINE)), cat="Binary")

    work = pulp.LpVariable.dicts("work", (t for t in range(DEADLINE)), cat="Binary")

    if (use_progress): 
        # This one will count up the seconds since each start, so we can calculate how which phase we are in
        work_time_progressed = pulp.LpVariable.dict("work_time_progressed", (t for t in range(DEADLINE)), lowBound=0, upBound=WORK_LENGTH, cat=pulp.LpInteger)

        # This one will count up the seconds since each start, so we can calculate how which phase we are in
        startup_time_progressed = pulp.LpVariable.dict("startup_time_progressed", (t for t in range(DEADLINE)), lowBound=0, upBound=STARTUP_LENGTH, cat=pulp.LpInteger)

        # set time_progressed to 0, whenever we start
        for t in range(DEADLINE-1):
            #https://download.aimms.com/aimms/download/manuals/AIMMS3OM_IntegerProgrammingTricks.pdf 
            if (t>0):
                # be bigger than the previous value IF starting
                prob += startup_time_progressed[t] >= startup_time_progressed[t-1] + 1 - (1 - starting[t]) * M
                prob += startup_time_progressed[t] <= startup_time_progressed[t-1] + 1 + (1 - starting[t]) * M

            # IF not starting, be 0
            prob += startup_time_progressed[t] <= starting[t] * M 

            prob += work_time_progressed[0] == 0
            if (t > 0):
                prob += work_time_progressed[t] == work_time_progressed[t-1] + work[t]


    if (linearize):
        # we need to linearize our phases.
        # we do that by creating a boolean dict which will be true for each time the phase is active
        phases = mockPowerFunction.phases

        lin_function_dicts = { }

        running_index = 0

        for phase_key, phases_of_key in phases.items():
            duration = 0
            if len(phases_of_key) == 0:
                continue

            lin_function_dicts[phase_key] = { }

            progress_variable = startup_time_progressed if phase_key == 'startup' else work_time_progressed
            state_variable = starting if phase_key == 'startup' else work

            for phase in phases_of_key:

                phase_name = phase['name'] + str(running_index)
                running_index += 1
                phase_variable_lower = pulp.LpVariable.dict(phase_name + "_lower", (t for t in range(DEADLINE)), cat="Binary")
                phase_variable_upper = pulp.LpVariable.dict(phase_name + "_upper", (t for t in range(DEADLINE)), cat="Binary")
                phase_variable = pulp.LpVariable.dict(phase_name, (t for t in range(DEADLINE)), cat="Binary")
                lin_function_dicts[phase_key][phase_name] = { }
                lin_function_dicts[phase_key][phase_name]['variable'] = phase_variable
                lin_function_dicts[phase_key][phase_name]['upper'] = phase_variable_upper
                lin_function_dicts[phase_key][phase_name]['lower'] = phase_variable_lower
                lin_function_dicts[phase_key][phase_name]['power'] = phase['power']

                # bounds are [lower, upper) for each phase
                lower_bound = max(duration, 0) 
                upper_bound = duration + 1 + int(phase["duration"])

                print(f'{phase_name} must be between {lower_bound} and {upper_bound}')

                for t in range(DEADLINE):

                    #https://math.stackexchange.com/a/3260529 this is basically magic
                    # this activates the phase_variable within (lower, upper)

                    prob += progress_variable[t] - lower_bound <= M*phase_variable_lower[t]
                    prob += lower_bound - progress_variable[t] <= M*(1-phase_variable_lower[t])

                    prob += upper_bound - progress_variable[t] <= M*phase_variable_upper[t]
                    prob += progress_variable[t] - upper_bound <= M*(1-phase_variable_upper[t])

                    prob += phase_variable[t] >= phase_variable_lower[t] + phase_variable_upper[t] + state_variable[t] - 2
                    prob += phase_variable[t] <= phase_variable_lower[t]
                    prob += phase_variable[t] <= phase_variable_upper[t]
                    prob += phase_variable[t] <= state_variable[t]
                
                duration += int(phase["duration"])

        # our carbon cost is equal to each phase being active * its power * the amount of carbon per timeslot
        all_phase_variables_with_power = []

        for overarching_phase in lin_function_dicts.values():
            for phase_entry in overarching_phase.values():
                all_phase_variables_with_power.append((phase_entry['variable'], phase_entry['power']))

        def carbon_cost_at_timeslot(t: int):
            return reduce(lambda problem, phase_tuple: problem + phase_tuple[0][t] * phase_tuple[1] * carbon_cost_at_time[t], all_phase_variables_with_power, pulp.LpAffineExpression())

    if (linearize):
        prob += pulp.lpSum([carbon_cost_at_timeslot(t) for t in range(DEADLINE)]) 

    else:
        if (use_startup):
            prob += pulp.lpSum([starting[t] * carbon_cost_at_time[t] + work[t] * carbon_cost_at_time[t] for t in range(DEADLINE)]) 
        else: 
            prob += pulp.lpSum([work[t] * carbon_cost_at_time[t] for t in range(DEADLINE)]) 


    # spend enough time processing
    prob += pulp.lpSum(work[t] for t in range(STARTUP_LENGTH, DEADLINE)) == WORK_LENGTH
    prob += pulp.lpSum(work[t] for t in range(STARTUP_LENGTH)) == 0

    if (use_startup):

        for t in range(DEADLINE - 1):
            # Ensure the job undergoes the startup phase whenever it resumes
            # if [0 , 1], this will be 1
            # t1   t2
            # 0     0 => 0
            # 0     1 => 1 
            # 1     0 => -1 / 0
            prob += startup_finished[t] >= work[t + 1] - work[t]

            # we can not be in startup and work at the same time
            prob += startup_finished[t] + work[t] <= 1
            prob += starting[t] + work[t] <= 1


        for i in range(STARTUP_LENGTH - 1, DEADLINE):
            prob += pulp.lpSum([starting[i - j] for j in range(STARTUP_LENGTH)]) >= STARTUP_LENGTH * startup_finished[i], f"Contiguity_{i}"


        # The solution so far seems to take a really long time, let's also add a maximum amount of startups to hopefully reduce the search space
        prob += pulp.lpSum([startup_finished[j] for j in range(DEADLINE)]) <= 5, f"Max_starts"

    solver = pulp.GUROBI_CMD(timeLimit=timelimit)

    prob.solve(solver)

    print(f"Status: {pulp.LpStatus[prob.status]}")

    print(f"Job schedule:")
    for t in range(DEADLINE):
        if pulp.value(work[t]) is not None and pulp.value(work[t])  > 0:
            print(f"  Time {t}: Workin")
#
    #    if pulp.value(startup_finished[t]) is not None and pulp.value(startup_finished[t])  > 0:
    #        print(f"  Time {t}: Startup finished")
#
    #    if pulp.value(work_time_progressed[t]) is not None and pulp.value(work_time_progressed[t])  > 0:
    #        print(f"  Time {t}: Progress: {pulp.value(work_time_progressed[t])}")
#
    #    if pulp.value(work[t]) is not None and pulp.value(work[t])  > 0:
    #        print(f"  Time {t}: Processing")



WORK_LENGTH is 100
STARTUP_LENGTH is 20


In [15]:

from typing import List
import plotly.io as pio

color_map = {
    "Work": "red",
    "Starting": "green",
    "Startup finished": "lime",
    "Idle": "black"
}


def find_upper_lower_index_for_phases(bool_list: List[bool]):
    groups = []
    start_index = None

    for i, value in enumerate(bool_list):
        if value: 
            if start_index is None:
                start_index = i
        else:
            if start_index is not None:
                groups.append((start_index, i - 1)) 
                start_index = None

    if start_index is not None:
        groups.append((start_index, len(bool_list) - 1))

    # Output the result
    return groups

def make_carbon_trace_with_work_or_startup(targets: List[tuple]):
    # add the work or startup information to the carbon_trace

    graph_times = list(carbon_cost_at_time.keys())
    graph_carbon_costs = list(carbon_cost_at_time.values())
    category = ['Idle'] * len(graph_times)

    for variable, name in targets:
        for t in range(len(variable)):
            if pulp.value(variable[t]):
                category[t] = name

    fig_carbon = go.Figure()

    fig_carbon.add_trace(go.Scatter(x=graph_times, y=graph_carbon_costs, mode='lines', showlegend=False, line=dict(color='black', width=2)))


    marker_fig = px.scatter(x=graph_times, y=graph_carbon_costs, color=category, color_discrete_map={
                "Work": "red",
                "Starting": "green",
                "Startup finished": "lime",
                "Idle": "black"}
    )

    for trace in marker_fig.data:
        fig_carbon.add_trace(trace)


    fig_carbon.update_layout({ 
        'yaxis_title': 'Carbon', 
        'xaxis_title': 'Time slots'
    })
    
    return fig_carbon


def make_trace_from_variables(targets: List[tuple]):

    fig = go.Figure()

    for variable, name in targets:
        times = [t for t in range(len(variable))]
        values = [pulp.value(variable[t]) if pulp.value(variable[t]) is not None else None for t in range(len(variable))]

        to_remove = [i for i, value in enumerate(values) if value == 0.0]

        filtered_times = [value for index, value in enumerate(times) if index not in to_remove]
        filtered_values = [value for index, value in enumerate(values) if index not in to_remove]

        # the things you do for a cool plot

        fig.add_trace(go.Scatter(x=filtered_times, y=filtered_values, showlegend=False, mode='markers',
        marker=dict(
            color=color_map[name],
        )))

    return fig

common_styling = {
    'xaxis_title': 'Time slots'
}

def first_plot_work_only():
    """first plot should just show the timeslot were work is being done
    """
    define_lp_problem(False, False, False, False, 120)

    print(work)

    prototype_fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
    
    carbon_plot = make_carbon_trace_with_work_or_startup([(work, 'Work')])

    carbon_plot.show()

    pio.write_image(prototype_fig, file="lp_work.pdf", format='pdf')

    return prototype_fig

def second_plot_overhead_via_startup():
    """second plot includes overhead for every start
    """
    define_lp_problem(True, False, False, False, 120)

    carbon_plot = make_carbon_trace_with_work_or_startup([(work, 'Work'), (starting, 'Starting'), (startup_finished, 'Startup finished')])

    carbon_plot.show()

    #pio.write_image(prototype_fig, file="lp_overhead.pdf", format='pdf')



def third_plot_adding_progress_counters():
    """second plot includes overhead for every start
    """
    define_lp_problem(True, False, True, False, 120)

    prototype_fig = make_subplots(rows=3, cols=1, shared_xaxes=True)
    boolean_plot = make_carbon_trace_with_work_or_startup([(work, 'Work'), (starting, 'Starting'), (startup_finished, 'Startup finished')])
    
    integers_plot = make_trace_from_variables([(startup_time_progressed, 'Starting'), (work_time_progressed, 'Work')])

    prototype_fig.add_traces(integers_plot.data, rows=1, cols=1)
    prototype_fig.add_traces(boolean_plot.data, rows=2, cols=1)

    prototype_fig.update_layout({'yaxis1_title':'Progress', 'yaxis2_title': 'Carbon', 'xaxis2_title':'Time slots'})

    prototype_fig.show()

    pio.write_image(prototype_fig, file="lp_progress.pdf", format='pdf')




first_plot_work_only()
second_plot_overhead_via_startup()
third_plot_adding_progress_counters()


Set parameter Username
Set parameter TimeLimit to value 120
Set parameter LogFile to value "gurobi.log"
Using license file /opt/gurobi1103/gurobi.lic
Academic license - for non-commercial use only - expires 2025-08-12

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 23.10")
Copyright (c) 2024, Gurobi Optimization, LLC

Read LP format model from file /tmp/eede3a1cdc424c73a2c2390cc3f354b0-pulp.lp
Reading time = 0.00 seconds
OBJ: 2 rows, 250 columns, 250 nonzeros

CPU model: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 2 rows, 250 columns and 250 nonzeros
Model fingerprint: 0x880e2e80
Variable types: 0 continuous, 250 integer (250 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-02, 4e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+02]
Found heuristic solution: objective 11.9587300
P

Set parameter Username
Set parameter TimeLimit to value 120
Set parameter LogFile to value "gurobi.log"
Using license file /opt/gurobi1103/gurobi.lic
Academic license - for non-commercial use only - expires 2025-08-12

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 23.10")
Copyright (c) 2024, Gurobi Optimization, LLC

Read LP format model from file /tmp/c994c84735be45048f1de41b3dbe14c6-pulp.lp
Reading time = 0.01 seconds
OBJ: 981 rows, 750 columns, 7094 nonzeros

CPU model: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 981 rows, 750 columns and 7094 nonzeros
Model fingerprint: 0x05aff3b4
Variable types: 0 continuous, 750 integer (750 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [4e-02, 4e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 62 rows and 42 columns
Pr

Set parameter Username
Set parameter TimeLimit to value 120
Set parameter LogFile to value "gurobi.log"
Using license file /opt/gurobi1103/gurobi.lic
Academic license - for non-commercial use only - expires 2025-08-12

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 23.10")
Copyright (c) 2024, Gurobi Optimization, LLC

Read LP format model from file /tmp/911ab74f0009452fa042cc167437d11d-pulp.lp
Reading time = 0.04 seconds
OBJ: 2223 rows, 1248 columns, 10073 nonzeros

CPU model: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 2223 rows, 1248 columns and 10073 nonzeros
Model fingerprint: 0x9a18c33e
Variable types: 0 continuous, 1248 integer (750 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [4e-02, 4e-01]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 5e+02]
Presolve removed 555 rows and 288 c

In [19]:
def get_color(event_str: str) -> str:
    if ('Start' in event_str):
        return 'rgba(255, 0, 0, 0.2)' 
    if ('Download' in event_str):
        return 'rgba(0, 255, 0, 0.2)'
    if ('High' in event_str):
        return 'rgba(236, 39, 245, 0.2)'
    if ('Low' in event_str or "eval" in event_str): # regrets
        return 'rgba(245, 172, 39, 0.2)'
    return 'rgba(0, 0, 0, 1)'

def get_label(event_str: str) -> str:
    if ('Start' in event_str):
        return 'Start Python' 
    if ('Download' in event_str):
        return 'Download data'
    if ('Start train' in event_str):
        return 'prepare dataset'
    if ('High' in event_str):
        return 'High power'
    if ('Low' in event_str):
        return 'Low power'
    return event_str

def plot_with_states():

    define_lp_problem(True, True, True, True, 360)

    prototype_fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
    boolean_plot = make_carbon_trace_with_work_or_startup([(work, 'Work'), (starting, 'Starting'), (startup_finished, 'Startup finished')])
    
    integers_plot = make_trace_from_variables([(startup_time_progressed, 'Starting'), (work_time_progressed, 'Work')])

    phases_variables_with_name = [
        *[(value['variable'], key) for (key, value) in lin_function_dicts['startup'].items()],
        *[(value['variable'], key) for (key, value) in lin_function_dicts['work'].items()]
    ]

    prototype_fig.add_traces(integers_plot.data, rows=1, cols=1)
    prototype_fig.add_traces(boolean_plot.data, rows=2, cols=1)

    for variable, name in phases_variables_with_name:
        bools = [True if pulp.value(variable[t]) > 0 else False for t in range(len(variable))]
        groups = find_upper_lower_index_for_phases(bools)
        for start, end in groups:
            print(f"{start}, {end}, {name}")
            prototype_fig.add_shape(type='rect',
                name=name, layer="below",
                label=dict(text=get_label(name), textangle=-90),
                x0=start, y0=0, x1=end, y1=100,
                line=dict(color='rgba(0, 0, 0, 0)'),
                row=1, col=1, fillcolor=get_color(name)
            )

    prototype_fig.update_layout({'yaxis1_title':'Progress', 'yaxis2_title': 'Carbon', 'xaxis2_title': 'Time slots'})
    prototype_fig.show()

    pio.write_image(prototype_fig, file="lp_states.pdf", format='pdf')

plot_with_states()

Start Python0 must be between 0 and 11
Download Data1 must be between 10 and 21
High2 must be between 0 and 51
Low3 must be between 50 and 101
Set parameter Username
Set parameter TimeLimit to value 360
Set parameter LogFile to value "gurobi.log"
Using license file /opt/gurobi1103/gurobi.lic
Academic license - for non-commercial use only - expires 2025-08-12

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 23.10")
Copyright (c) 2024, Gurobi Optimization, LLC

Read LP format model from file /tmp/3f086895910d4b10a1cd9308f04598f6-pulp.lp
Reading time = 0.04 seconds
OBJ: 10223 rows, 4250 columns, 28073 nonzeros

CPU model: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 10223 rows, 4250 columns and 28073 nonzeros
Model fingerprint: 0x25d4afc1
Variable types: 0 continuous, 4250 integer (3750 binary)
Coefficient statistics:
  Matrix range     [1e+00

In [17]:
import sys

from scheduling.suspend_phases_scheduling_policy import SchedulerDebugOptions
sys.path.append('../src/')

import scheduling as gaia_sched
import carbon
import power_consumption_profiles as pcp

carbon_trace = carbon.get_carbon_model("AU-SA", 7000, extra_columns=True).extend(3600)
scheduler = gaia_sched.SuspendSchedulingDynamicPowerPolicy(None, carbon_trace)
options = SchedulerDebugOptions(
        use_startup=True, 
        dynamic_power=True, 
        use_progress=True, 
        linearize=True,
        timelimit=None,
        scale_time=True,
)

foo_model = pcp.PeriodicPowerFunction({
    'startup':[{'name': 'startup','duration': 0, 'power': 150}],
    'work':[{'name': 'high', 'power': 200, 'duration': 3600}, {'name': 'low', 'power': 100, 'duration': 3600}]
}, length=3*60*30)

test_model = pcp.PeriodicPowerFunction({
    'startup':[{'name': 'startup','duration': 0, 'power': 200}],
    'work':[{'name': 'high', 'power': 200, 'duration': 1800}, {'name': 'low', 'power': 100, 'duration': 3600}]
}, length=9600)

debug_result = scheduler.find_execution_times(carbon_trace, 4*60*60, test_model, debugOptions=options)
real_result = scheduler.find_execution_times(carbon_trace, 4*60*60, test_model)

scaled deadlines is: 24, spt: 600
Set parameter Username
Set parameter LogFile to value "gurobi.log"
Set parameter Threads to value 256
Using license file /opt/gurobi1103/gurobi.lic
Academic license - for non-commercial use only - expires 2025-08-12

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 23.10")
Copyright (c) 2024, Gurobi Optimization, LLC

Read LP format model from file /tmp/63f49f9fe81c4c14a001d7eed202e62d-pulp.lp
Reading time = 0.00 seconds
OBJ: 865 rows, 384 columns, 2010 nonzeros

CPU model: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 256 threads

         Reduce the value of the Threads parameter to improve performance


Optimize a model with 865 rows, 384 columns and 2010 nonzeros
Model fingerprint: 0x60fff56a
Variable types: 1 continuous, 383 integer (359 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [3e-03, 6e-03]


In [18]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import pulp

print(debug_result)
print(real_result)
print(len(real_result))

def plot_scaled_time_optimization():
    debug_graph_times = list(debug_result['carbon_trace'].keys())
    debug_graph_carbon_costs = list(debug_result['carbon_trace'].values())
    debug_fig_carbon = px.scatter(x=debug_graph_times, y=debug_graph_carbon_costs)

    prototype_fig = make_subplots(rows=4, cols=1, shared_xaxes=True)
    boolean_plot = make_trace_from_variables([(debug_result['work'], 'work'), (debug_result['starting'], 'starting'), (debug_result['startup_finished'], 'startup_finished')])
    
    integers_plot = make_trace_from_variables([(debug_result['startup_time_progressed'], 'startup_time_progressed'), (debug_result['work_time_progressed'], 'work_time_progressed')])

    phases_variables_with_name = [
        *[(value['variable'], key) for (key, value) in debug_result['lin_function_dicts']['startup'].items()],
        *[(value['variable'], key) for (key, value) in debug_result['lin_function_dicts']['work'].items()]
    ]

    states_plot = make_trace_from_variables(phases_variables_with_name)

    prototype_fig.add_traces(integers_plot.data, rows=1, cols=1)
    prototype_fig.add_traces(boolean_plot.data, rows=2, cols=1)
    prototype_fig.add_traces(states_plot.data, rows=3, cols=1)
    prototype_fig.add_traces(debug_fig_carbon.data[0], rows=4, cols=1)

    prototype_fig.update_layout({'yaxis1_title':'Progress', 'yaxis2_title':'States', 'yaxis3_title':'Phases', 'yaxis4_title': 'Carbon', 'xaxis4_title': 'Time slots'})


    prototype_fig.show()

plot_scaled_time_optimization()


{'carbon_trace': {0: 2.6705555555555555e-05, 1: 2.6705555555555555e-05, 2: 2.6705555555555555e-05, 3: 2.6705555555555555e-05, 4: 2.6705555555555555e-05, 5: 2.6705555555555555e-05, 6: 2.753611111111111e-05, 7: 2.753611111111111e-05, 8: 2.753611111111111e-05, 9: 2.753611111111111e-05, 10: 2.753611111111111e-05, 11: 2.753611111111111e-05, 12: 2.8436111111111113e-05, 13: 2.8436111111111113e-05, 14: 2.8436111111111113e-05, 15: 2.8436111111111113e-05, 16: 2.8436111111111113e-05, 17: 2.8436111111111113e-05, 18: 2.8641666666666664e-05, 19: 2.8641666666666664e-05, 20: 2.8641666666666664e-05, 21: 2.8641666666666664e-05, 22: 2.8641666666666664e-05, 23: 2.8641666666666664e-05}, 'starting': {0: starting_0, 1: starting_1, 2: starting_2, 3: starting_3, 4: starting_4, 5: starting_5, 6: starting_6, 7: starting_7, 8: starting_8, 9: starting_9, 10: starting_10, 11: starting_11, 12: starting_12, 13: starting_13, 14: starting_14, 15: starting_15, 16: starting_16, 17: starting_17, 18: starting_18, 19: start

KeyError: 'work'